# This is Tutorial for the Camera Calibration using OpenCV from link
'https://docs.opencv.org/3.4/dc/dbb/tutorial_py_calibration.html'

# Procedure for Camera Calibration:

1. Capture more images to calibrate camera more precisely.
2. In this tutorial we have captured ~300 images

# Guidlines to Capture Images:

1. Take the chessboard and place it on any corner of the setup
2. Move the chessboard all over the setup and capture images
3. Capture images at different height and angles (The board angle should not be more than 45°)

# Steps(By using docker file) ;
 
  1. Build the DockerImage by running: $sudo docker build ./ --tag  bfs-image-publisher (without calibration.yaml)
  
  2. Run the DockerImage by running:  $sudo docker run -t --privileged -v /dqev/bus/usb:/dev/bus/usb -p 5557:5557 -p 5556:5556 bfs-      image-publisher
  
  3. Run the Subscriber code in conda environment using: $python3 camerasubscriber.py and RUN python3 calibration.py --     save_img.true in the command line to capture the images
  
  4. Collect the images by placing the target at different positions and angles
  
  5. follow the next cells

# Example of carrera setup (Original Image and Calibrated)
<table><tr>
<td> <img src="Original.png" title='Original' style="width: 280px;"/> </td>
<td> <img src="calibrated.png" style="width: 280px;"/> </td>
</tr></table>

# Importing Libraries

In [ ]:
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt
import yaml

# Defining object points, criteria and Reading captured images

In [ ]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Defining the world coordinates for 3D points
objp = np.zeros((7*10,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:10].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('../subscriber/data/img/temp_img_data/*.png')
#print(images)

# Extracting path of individual image stored in a given directory

In [ ]:
for fname in images:
    img = cv.imread(fname)
    #img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)# TODO COLOR
    gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)# TODO COLOR
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,10), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,10), corners2, ret)

# Performing camera calibration

The mtx and dst values are stored in yaml file for the calibration

In [ ]:
"""
Performing camera calibration by passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(mtx)
print(dist)

# Creation of config file for camera calibration

In [ ]:
# yaml code implementation
with open('../bfs-camera-driver/publisher/calibration.yaml') as fh:
    read_data = yaml.load(fh, Loader = yaml.FullLoader)
    
    # Reading Elementss from the yaml file
    matx = read_data['camera_matrix']
    dist_coeff = read_data['dist_coeff']
    
    # Printing yaml file values
    print('Camera Matrix ; ', matx)
    print('Distance Coefficients : ', dist_coeff)

# Steps( By using yaml file)
1. Create a yaml file which contains the value of camera matrix and distance coefficients obtsined from 'mtx' and 'dst' in code below.

2. Run the publisher code in conda environment using $python3 camerapublisher.py.

3. Run the Subscriber code in conda environment using $python3 camerasubscriber.py. 

# Test Results: Reading image to undisort it
The function 'cv.getOptimalNewCameraMatrix' computes and returns the optimal new camera intrinsic matrix based on the free scaling parameter

cv.getOptimalNewCameraMatrix(cameraMatrix, distortion Coefficient, imgSize, alpha=1, newImgSize(should be equal to imgSize))

In [ ]:
img = cv.imread('Original.png')
w, h = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

print(newcameramtx)

# Undistortion Method 1

In [ ]:
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi

with open('../bfs-camera-driver/publisher/camera_calibration.yaml') as fh:
    read_data = yaml.load(fh, Loader = yaml.FullLoader)
    
    # Cropping Image
    x_max = read_data['x_start']
    y_max = read_data['y_start']
    x_min = read_data['w']
    y_min = read_data['h']

    dst1 = dst[y_max:y+y_min, x_max:x+x_min]

    
# Writing image to directory
cv.imwrite('calibrated-image.png', dst1)

# plotting original and calibrated image
plt.subplot(121)
plt.imshow(img)
plt.title('Original Image')
plt.subplot(122)
plt.imshow(dst1)
plt.title('Calibrated Image')

# Undistortion Method 2

In [ ]:
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w+x, h+y), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi

y_start = 225
x_start = 300
w = 900
h = 670

dst1 = dst[y_start:y+h, x_start:x+w]

# plotting original and calibrated image
plt.subplot(121)
plt.imshow(img)
plt.title('Original Image')
plt.subplot(122)
plt.imshow(dst1)
plt.title('Calibrated Image')